In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# 1. BOOLEAN RETRIEVAL MODEL

In [ ]:
import re
from collections import defaultdict
from typing import Dict, List, Set, Union
class BooleanRetrieval:
    def __init__(self):
        self.documents = {}  # doc_id -> document text
        self.inverted_index = defaultdict(set)  # term -> set of doc_ids
        
    def add_document(self, doc_id: str, text: str):
        """Add a document to the collection."""
        self.documents[doc_id] = text
        # Tokenize and add to inverted index
        tokens = self._tokenize(text.lower())
        for token in tokens:
            self.inverted_index[token].add(doc_id)
    
    def _tokenize(self, text: str) -> List[str]:
        """Simple tokenization - split on whitespace and remove punctuation."""
        # Remove punctuation and split
        text = re.sub(r'[^\w\s]', ' ', text)
        return text.split()
    
    def _get_postings(self, term: str) -> Set[str]:
        """Get the posting list (document IDs) for a term."""
        return self.inverted_index.get(term.lower(), set())
    
    def boolean_and(self, term1: str, term2: str) -> Set[str]:
        """Return documents containing both terms."""
        return self._get_postings(term1) & self._get_postings(term2)
    
    def boolean_or(self, term1: str, term2: str) -> Set[str]:
        """Return documents containing either term."""
        return self._get_postings(term1) | self._get_postings(term2)
    
    def boolean_not(self, term: str) -> Set[str]:
        """Return documents NOT containing the term."""
        all_docs = set(self.documents.keys())
        return all_docs - self._get_postings(term)
    
    def query(self, query_string: str) -> Set[str]:
        """
        Process a boolean query string.
        Supports: AND, OR, NOT, parentheses
        Example: "python AND (machine OR learning) AND NOT tutorial"
        """
        return self._parse_query(query_string)
    
    def _parse_query(self, query: str) -> Set[str]:
        """Parse and evaluate a boolean query."""
        # Tokenize the query
        tokens = self._tokenize_query(query)
        
        # Convert to postfix notation (Reverse Polish Notation)
        postfix = self._infix_to_postfix(tokens)
        
        # Evaluate postfix expression
        return self._evaluate_postfix(postfix)
    
    def _tokenize_query(self, query: str) -> List[str]:
        """Tokenize a boolean query, preserving operators and parentheses."""
        # Replace operators with standardized versions
        query = re.sub(r'\bAND\b', 'AND', query, flags=re.IGNORECASE)
        query = re.sub(r'\bOR\b', 'OR', query, flags=re.IGNORECASE) 
        query = re.sub(r'\bNOT\b', 'NOT', query, flags=re.IGNORECASE)
        
        # Split on whitespace but keep operators and parentheses
        tokens = []
        for token in query.split():
            if token in ['AND', 'OR', 'NOT', '(', ')']:
                tokens.append(token)
            else:
                # Remove parentheses attached to terms
                token = token.strip('()')
                if token:
                    tokens.append(token.lower())
        
        return tokens
    
    def _infix_to_postfix(self, tokens: List[str]) -> List[str]:
        """Convert infix boolean expression to postfix notation."""
        precedence = {'NOT': 3, 'AND': 2, 'OR': 1}
        stack = []
        postfix = []
        
        for token in tokens:
            if token in precedence:
                while (stack and stack[-1] != '(' and 
                       stack[-1] in precedence and
                       precedence[stack[-1]] >= precedence[token]):
                    postfix.append(stack.pop())
                stack.append(token)
            elif token == '(':
                stack.append(token)
            elif token == ')':
                while stack and stack[-1] != '(':
                    postfix.append(stack.pop())
                if stack:
                    stack.pop()  # Remove the '('
            else:
                postfix.append(token)  # It's a term
        
        while stack:
            postfix.append(stack.pop())
        
        return postfix
    
    def _evaluate_postfix(self, postfix: List[str]) -> Set[str]:
        """Evaluate a postfix boolean expression."""
        stack = []
        
        for token in postfix:
            if token == 'AND':
                if len(stack) >= 2:
                    b = stack.pop()
                    a = stack.pop()
                    stack.append(a & b)
            elif token == 'OR':
                if len(stack) >= 2:
                    b = stack.pop()
                    a = stack.pop()
                    stack.append(a | b)
            elif token == 'NOT':
                if len(stack) >= 1:
                    a = stack.pop()
                    all_docs = set(self.documents.keys())
                    stack.append(all_docs - a)
            else:
                # It's a term
                stack.append(self._get_postings(token))
        
        return stack[0] if stack else set()
    
    def get_results_with_text(self, doc_ids: Set[str]) -> Dict[str, str]:
        """Return document IDs with their text content."""
        return {doc_id: self.documents[doc_id] for doc_id in doc_ids}
    

In [ ]:
# document
doc =  {"d1": "Belgium is famous for chocolate, beer and waffle.",
        "d2":"Have you ever had Belgian fries?",
        "d3":"I have lived in Leuven for 4 years.",
        "d4":"I never liked chocolate on my waffle.",
        "d5":"Let's go out for some beers tonight."}

# vocab = [chocolate, beer, waffle, fries]

In [11]:
br = BooleanRetrieval() 
for doc_id, text in doc.items():
    br.add_document(doc_id, text)

results = br.query("waffle and chocolate")
print(f"Results: {sorted(results)}")

Results: ['d1', 'd4']


# 2. TF-IDF

In [12]:
from collections import Counter
import pandas as pd
import math
import numpy as np
import string
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Pierre-
[nltk_data]     François\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
data =  ["Belgium is famous for chocolate, beer and waffle.",
        "Have you ever had Belgian fries?",
        "I have lived in Leuven for 4 years.",
        "I never liked chocolate on my waffle.",
        "Let's go out for some beers tonight."]

In [14]:
def preprocess(list_of_text):
  preprocessed_text = []
  for text in list_of_text:
    text = text.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    text = text.lower() #convert to lower case
    words = word_tokenize(text) #tokenize the text
    preprocessed_text.append(words)
  return preprocessed_text

preprocessed_text = preprocess(data)

In [15]:
print(preprocessed_text[0])
print(preprocessed_text[1])
print(preprocessed_text[2])
print(preprocessed_text[3])
print(preprocessed_text[4])

['belgium', 'is', 'famous', 'for', 'chocolate', 'beer', 'and', 'waffle']
['have', 'you', 'ever', 'had', 'belgian', 'fries']
['i', 'have', 'lived', 'in', 'leuven', 'for', '4', 'years']
['i', 'never', 'liked', 'chocolate', 'on', 'my', 'waffle']
['lets', 'go', 'out', 'for', 'some', 'beers', 'tonight']


In [19]:
term_frequency = []
for term in preprocessed_text:
    word_count = Counter(term)
    term_frequency.append(word_count)

print(term_frequency)

term_frequency = pd.DataFrame.from_dict(term_frequency,orient="columns")
term_frequency = term_frequency.fillna(0)
term_frequency

[Counter({'belgium': 1, 'is': 1, 'famous': 1, 'for': 1, 'chocolate': 1, 'beer': 1, 'and': 1, 'waffle': 1}), Counter({'have': 1, 'you': 1, 'ever': 1, 'had': 1, 'belgian': 1, 'fries': 1}), Counter({'i': 1, 'have': 1, 'lived': 1, 'in': 1, 'leuven': 1, 'for': 1, '4': 1, 'years': 1}), Counter({'i': 1, 'never': 1, 'liked': 1, 'chocolate': 1, 'on': 1, 'my': 1, 'waffle': 1}), Counter({'lets': 1, 'go': 1, 'out': 1, 'for': 1, 'some': 1, 'beers': 1, 'tonight': 1})]


,belgium,is,famous,for,chocolate,beer,and,waffle,have,you,...,never,liked,on,my,lets,go,out,some,beers,tonight
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [31]:
from nltk.corpus import stopwords

def stopword_removal(list_of_text):
    remove_stopword = []
    for text in list_of_text:
        text = text.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
        text = text.lower() #convert to lower case
        words = word_tokenize(text) #tokenize the text
        words = [word for word in words if word not in stopwords.words('english')] #remove stopword
        remove_stopword.append(words)
    return remove_stopword

preprocessed_text_new = stopword_removal(data)
term_frequency_new = []
for term in preprocessed_text_new:
    word_count = Counter(term)
    term_frequency_new.append(word_count)

term_frequency_new = pd.DataFrame.from_dict(term_frequency_new,"columns")
term_frequency_new = term_frequency_new.fillna(0)
term_frequency_new

,belgium,famous,chocolate,beer,waffle,ever,belgian,fries,lived,leuven,4,years,never,liked,lets,go,beers,tonight
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [32]:
idf = {}
for word in term_frequency_new.columns:
  idf[word] = math.log(len(preprocessed_text)/term_frequency[word].apply(lambda x: 1 if x > 0 else 0).sum())

idf = pd.DataFrame.from_dict(idf, orient="index").transpose()
idf

,belgium,famous,chocolate,beer,waffle,ever,belgian,fries,lived,leuven,4,years,never,liked,lets,go,beers,tonight
0,1.609438,1.609438,0.916291,1.609438,0.916291,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438


In [34]:
tfidf_dict = {}
for i in range(len(preprocessed_text_new)):
    documents_tfidf = []
    for column in idf:
        tfidf = term_frequency_new[column][i]*idf[column]
        documents_tfidf.append(tfidf)
    tfidf_dict[f"Document {str(i)}"] = documents_tfidf

tfidf_df = pd.DataFrame.from_dict(tfidf_dict,orient="index",columns=idf.columns)
tfidf_df = tfidf_df.round(2)
tfidf_df

,belgium,famous,chocolate,beer,waffle,ever,belgian,fries,lived,leuven,4,years,never,liked,lets,go,beers,tonight
Document 0,"0 1.609438 Name: belgium, dtype: float64","0 1.609438 Name: famous, dtype: float64","0 0.916291 Name: chocolate, dtype: float64","0 1.609438 Name: beer, dtype: float64","0 0.916291 Name: waffle, dtype: float64","0 0.0 Name: ever, dtype: float64","0 0.0 Name: belgian, dtype: float64","0 0.0 Name: fries, dtype: float64","0 0.0 Name: lived, dtype: float64","0 0.0 Name: leuven, dtype: float64","0 0.0 Name: 4, dtype: float64","0 0.0 Name: years, dtype: float64","0 0.0 Name: never, dtype: float64","0 0.0 Name: liked, dtype: float64","0 0.0 Name: lets, dtype: float64","0 0.0 Name: go, dtype: float64","0 0.0 Name: beers, dtype: float64","0 0.0 Name: tonight, dtype: float64"
Document 1,"0 0.0 Name: belgium, dtype: float64","0 0.0 Name: famous, dtype: float64","0 0.0 Name: chocolate, dtype: float64","0 0.0 Name: beer, dtype: float64","0 0.0 Name: waffle, dtype: float64","0 1.609438 Name: ever, dtype: float64","0 1.609438 Name: belgian, dtype: float64","0 1.609438 Name: fries, dtype: float64","0 0.0 Name: lived, dtype: float64","0 0.0 Name: leuven, dtype: float64","0 0.0 Name: 4, dtype: float64","0 0.0 Name: years, dtype: float64","0 0.0 Name: never, dtype: float64","0 0.0 Name: liked, dtype: float64","0 0.0 Name: lets, dtype: float64","0 0.0 Name: go, dtype: float64","0 0.0 Name: beers, dtype: float64","0 0.0 Name: tonight, dtype: float64"
Document 2,"0 0.0 Name: belgium, dtype: float64","0 0.0 Name: famous, dtype: float64","0 0.0 Name: chocolate, dtype: float64","0 0.0 Name: beer, dtype: float64","0 0.0 Name: waffle, dtype: float64","0 0.0 Name: ever, dtype: float64","0 0.0 Name: belgian, dtype: float64","0 0.0 Name: fries, dtype: float64","0 1.609438 Name: lived, dtype: float64","0 1.609438 Name: leuven, dtype: float64","0 1.609438 Name: 4, dtype: float64","0 1.609438 Name: years, dtype: float64","0 0.0 Name: never, dtype: float64","0 0.0 Name: liked, dtype: float64","0 0.0 Name: lets, dtype: float64","0 0.0 Name: go, dtype: float64","0 0.0 Name: beers, dtype: float64","0 0.0 Name: tonight, dtype: float64"
Document 3,"0 0.0 Name: belgium, dtype: float64","0 0.0 Name: famous, dtype: float64","0 0.916291 Name: chocolate, dtype: float64","0 0.0 Name: beer, dtype: float64","0 0.916291 Name: waffle, dtype: float64","0 0.0 Name: ever, dtype: float64","0 0.0 Name: belgian, dtype: float64","0 0.0 Name: fries, dtype: float64","0 0.0 Name: lived, dtype: float64","0 0.0 Name: leuven, dtype: float64","0 0.0 Name: 4, dtype: float64","0 0.0 Name: years, dtype: float64","0 1.609438 Name: never, dtype: float64","0 1.609438 Name: liked, dtype: float64","0 0.0 Name: lets, dtype: float64","0 0.0 Name: go, dtype: float64","0 0.0 Name: beers, dtype: float64","0 0.0 Name: tonight, dtype: float64"
Document 4,"0 0.0 Name: belgium, dtype: float64","0 0.0 Name: famous, dtype: float64","0 0.0 Name: chocolate, dtype: float64","0 0.0 Name: beer, dtype: float64","0 0.0 Name: waffle, dtype: float64","0 0.0 Name: ever, dtype: float64","0 0.0 Name: belgian, dtype: float64","0 0.0 Name: fries, dtype: float64","0 0.0 Name: lived, dtype: float64","0 0.0 Name: leuven, dtype: float64","0 0.0 Name: 4, dtype: float64","0 0.0 Name: years, dtype: float64","0 0.0 Name: never, dtype: float64","0 0.0 Name: liked, dtype: float64","0 1.609438 Name: lets, dtype: float64","0 1.609438 Name: go, dtype: float64","0 1.609438 Name: beers, dtype: float64","0 1.609438 Name: tonight, dtype: float64"


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english")

analyze = vectorizer.build_analyzer()

print("Document 1", analyze(data[0]))
print("Document 2", analyze(data[1]))

Document 1 ['belgium', 'famous', 'chocolate', 'beer', 'waffle']
Document 2 ['belgian', 'fries']


In [65]:
X = vectorizer.fit_transform(data)
tfidf = pd.DataFrame(X.toarray(), index = range(len(data)),columns=vectorizer.get_feature_names_out())
tfidf = tfidf.round(2)
tfidf

,beer,beers,belgian,belgium,chocolate,famous,fries,let,leuven,liked,lived,tonight,waffle,years
0,0.48,0.00,0.00,0.48,0.39,0.48,0.00,0.00,0.00,0.00,0.00,0.00,0.39,0.00
1,0.00,0.00,0.71,0.00,0.00,0.00,0.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.58,0.00,0.58,0.00,0.00,0.58
3,0.00,0.00,0.00,0.00,0.53,0.00,0.00,0.00,0.00,0.66,0.00,0.00,0.53,0.00
4,0.00,0.58,0.00,0.00,0.00,0.00,0.00,0.58,0.00,0.00,0.00,0.58,0.00,0.00


In [66]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def preprocess(doc):   #this takes in a string and converts it into a list
  doc = doc.split()
  preprocessed_text = []
  for text in doc:
    text = text.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    text = text.lower() #convert to lower case
    words = word_tokenize(text) #tokenize the text
    words = [word for word in words if word not in stopwords.words('english')] #remove stopwords
    words = [lemmatizer.lemmatize(word) for word in words] #lemmatize
    if words != []:
      preprocessed_text.append(words[0])
  return preprocessed_text


In [67]:
vectorizer = TfidfVectorizer(tokenizer=preprocess)
analyze = vectorizer.build_analyzer()
print("Document 1", analyze(data[0]))
print("Document 2", analyze(data[1]))
print("Document 3", analyze(data[2]))
print("Document 4", analyze(data[3]))
print("Document 5", analyze(data[4]))

Document 1 ['belgium', 'famous', 'chocolate', 'beer', 'waffle']
Document 2 ['ever', 'belgian', 'fry']
Document 3 ['lived', 'leuven', '4', 'year']
Document 4 ['never', 'liked', 'chocolate', 'waffle']
Document 5 ['let', 'go', 'beer', 'tonight']


In [68]:
X = vectorizer.fit_transform(data)
tfidf_df = pd.DataFrame(X.toarray(), index=range(len(data)), columns=vectorizer.get_feature_names_out())
tfidf_df = tfidf_df.round(2)
tfidf_df

c:\Users\Pierre-François\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,4,beer,belgian,belgium,chocolate,ever,famous,fry,go,let,leuven,liked,lived,never,tonight,waffle,year
0,0.0,0.41,0.00,0.5,0.41,0.00,0.5,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.41,0.0
1,0.0,0.00,0.58,0.0,0.00,0.58,0.0,0.58,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0
2,0.5,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.5,0.00,0.5,0.00,0.00,0.00,0.5
3,0.0,0.00,0.00,0.0,0.44,0.00,0.0,0.00,0.00,0.00,0.0,0.55,0.0,0.55,0.00,0.44,0.0
4,0.0,0.42,0.00,0.0,0.00,0.00,0.0,0.00,0.52,0.52,0.0,0.00,0.0,0.00,0.52,0.00,0.0


In [73]:
query = "beer and fries"
query = preprocess(query)
print(query)

['beer', 'fry']


In [74]:
query_vector = vectorizer.transform([" ".join(query)]) #the vectorizer takes as input a list of queries - so here it will take ["beer fry"] as query. You can pass multiple queries as ["beer fry", "waffle fry"] which will return one vector for each query
print(query_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (1, 17)>
  Coords	Values
  (0, 1)	0.6279137616509933
  (0, 7)	0.7782829228046183


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(query_vector, X)

In [72]:
results = [(data[i], cosine_similarities[0][i]) for i in range(len(data))]
results.sort(key=lambda x: x[1], reverse=True)
for doc, similarity in results:
    print(f"Similarity: {similarity:.2f}\n{doc}\n")

Similarity: 0.45
Have you ever had Belgian fries?

Similarity: 0.27
Let's go out for some beers tonight.

Similarity: 0.25
Belgium is famous for chocolate, beer and waffle.

Similarity: 0.00
I have lived in Leuven for 4 years.

Similarity: 0.00
I never liked chocolate on my waffle.



In [78]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
preprocessed_text = [preprocess(doc) for doc in data]
bm25 = BM25Okapi(preprocessed_text) #fit the model to our data

In [80]:
scores = bm25.get_scores(query) #this returns an array of scores, where we can map the index to the index of the data to get the ranking

score_to_id = {}
for i in range(len(scores)):
  score_to_id[str(i)] = float(scores[i])

ranking = sorted(score_to_id.items(), key=lambda x:x[1], reverse=True)

for k,v in ranking[:5]:
  print(f'Document: {data[int(k)]}\n Score: {v}')


Document: Have you ever had Belgian fries?
 Score: 1.23787300131618
Document: Let's go out for some beers tonight.
 Score: 0.33647223662121295
Document: Belgium is famous for chocolate, beer and waffle.
 Score: 0.30244695426625884
Document: I have lived in Leuven for 4 years.
 Score: 0.0
Document: I never liked chocolate on my waffle.
 Score: 0.0


In [86]:
query = "chocolate and waffle and beer"
query = preprocess(query)
print(query)

scores = bm25.get_scores(query) #this returns an array of scores, where we can map the index to the index of the data to get the ranking

score_to_id = {}
for i in range(len(scores)):
  score_to_id[str(i)] = float(scores[i])

ranking = sorted(score_to_id.items(), key=lambda x:x[1], reverse=True)

for k,v in ranking[:5]:
  print(f'Document: {data[int(k)]}\n Score: {v}')


['chocolate', 'waffle', 'beer']
Document: Belgium is famous for chocolate, beer and waffle.
 Score: 0.9073408627987765
Document: I never liked chocolate on my waffle.
 Score: 0.6729444732424259
Document: Let's go out for some beers tonight.
 Score: 0.33647223662121295
Document: Have you ever had Belgian fries?
 Score: 0.0
Document: I have lived in Leuven for 4 years.
 Score: 0.0


In [87]:
!pip install datasets
from datasets import load_dataset


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ------ --------------------------------- 4.5/25.8 MB 22.3 MB/s eta 0:00:01
   -------------- ------------------------- 9.4/25.8 MB 23.5 MB/s eta 0:00:01
   ---------------------- ----------------- 14.7/25.8 MB 24.3 MB/s eta 0:00:01
   ------------------------------ --------- 19.7/25.8 MB 23.4 MB/s eta 0:00:01
   ---------------------------------------  25.2/25.8 MB 24.1 MB/s eta 0:00:01
   ---------------------------------------- 25.8/25.8 MB 22.7 MB/s eta 0:00:00

   -- -------------------------------------  1/15 [pyarrow]
   -- -------------------------------------  1/15 [pyarrow]
   -- -------------------------------------  1/15 [pyarrow]
   -- -------------------------------------  1/15 [pyarrow]
   -- -------------------------------------  1/15 [pyarrow]
   -- -------------------------------------  1/15 [pyarrow]
   -- -------------------------------------  1/15 [pyarrow]
   -- -------------------------------

c:\Users\Pierre-François\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [88]:
data = load_dataset("wikipedia", "20220301.simple", trust_remote_code=True)['train']
data = data.select(range(200))

c:\Users\Pierre-François\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pierre-François\.cache\huggingface\hub\datasets--wikipedia. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 205328/205328 [00:00<00:00, 253648.68 exam

In [98]:
print(data['text'][0])

April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.

April always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.

April's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.

The Month 

April comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.

April begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.

In common years, April starts on the same day of the week as October of the previous year, and in leap years, M

In [ ]:
query_set = [
    "Earth's atmosphere",
    "Agricultural crops",
    "Parts of the human body",
    "What are the official languages of countries?",
    "Best places to travel"]

In [104]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import math
import re
from typing import List, Dict, Tuple, Set
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [107]:
dataset = load_dataset("wikipedia", "20220301.simple", trust_remote_code=True)['train']
subset_size = 200
wiki_data = dataset.select(range(subset_size))

In [109]:
documents = []
titles = []
for item in wiki_data:
    documents.append(item['text'])
    titles.append(item['title'])
print(f"Loaded {len(documents)} documents")

Loaded 200 documents


In [108]:
print(data['text'][0])

April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.

April always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.

April's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.

The Month 

April comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.

April begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.

In common years, April starts on the same day of the week as October of the previous year, and in leap years, M

In [113]:
class TextProcessor:
    """Text preprocessing utilities"""
    
    def __init__(self, use_stopwords=True, use_lemmatization=True):
        self.use_stopwords = use_stopwords
        self.use_lemmatization = use_lemmatization
        
        if use_stopwords:
            self.stop_words = set(stopwords.words('english'))
        else:
            self.stop_words = set()
            
        if use_lemmatization:
            self.lemmatizer = WordNetLemmatizer()
        else:
            self.lemmatizer = None
    
    def preprocess_text(self, text: str) -> List[str]:
        """Preprocess text: tokenize, lowercase, remove stopwords, lemmatize"""
        # Convert to lowercase and tokenize
        text = text.lower()
        tokens = word_tokenize(text)
        
        # Remove non-alphabetic tokens and stopwords
        tokens = [token for token in tokens if token.isalpha()]
        
        if self.use_stopwords:
            tokens = [token for token in tokens if token not in self.stop_words]
        
        # Lemmatize
        if self.lemmatizer:
            tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        
        return tokens

In [114]:
class BooleanModel:
    """Boolean Information Retrieval Model"""
    
    def __init__(self, documents: List[str], processor: TextProcessor):
        self.documents = documents
        self.processor = processor
        self.inverted_index = self._build_inverted_index()
    
    def _build_inverted_index(self) -> Dict[str, Set[int]]:
        """Build inverted index for boolean retrieval"""
        index = defaultdict(set)
        
        for doc_id, doc in enumerate(self.documents):
            tokens = self.processor.preprocess_text(doc)
            for token in set(tokens):  # Use set to avoid duplicates
                index[token].add(doc_id)
        
        return dict(index)
    
    def search(self, query: str) -> List[int]:
        """Boolean search with AND operation"""
        query_terms = self.processor.preprocess_text(query)
        
        if not query_terms:
            return []
        
        # Start with documents containing the first term
        result_docs = self.inverted_index.get(query_terms[0], set()).copy()
        
        # Intersect with documents containing other terms (AND operation)
        for term in query_terms[1:]:
            term_docs = self.inverted_index.get(term, set())
            result_docs = result_docs.intersection(term_docs)
        
        return list(result_docs)


In [115]:
class VectorSpaceModel:
    """Vector Space Model with TF-IDF"""
    
    def __init__(self, documents: List[str], processor: TextProcessor, 
                 similarity_measure='cosine'):
        self.documents = documents
        self.processor = processor
        self.similarity_measure = similarity_measure
        
        # Preprocess documents
        processed_docs = []
        for doc in documents:
            tokens = processor.preprocess_text(doc)
            processed_docs.append(' '.join(tokens))
        
        # Create TF-IDF vectors
        self.vectorizer = TfidfVectorizer()
        self.doc_vectors = self.vectorizer.fit_transform(processed_docs)
    
    def search(self, query: str, top_k: int = 10) -> List[Tuple[int, float]]:
        """Search using vector space model"""
        # Preprocess query
        query_tokens = self.processor.preprocess_text(query)
        query_text = ' '.join(query_tokens)
        
        # Convert query to vector
        query_vector = self.vectorizer.transform([query_text])
        
        # Calculate similarities
        if self.similarity_measure == 'cosine':
            similarities = cosine_similarity(query_vector, self.doc_vectors).flatten()
        elif self.similarity_measure == 'euclidean':
            # For Euclidean, we use negative distance (smaller distance = higher similarity)
            from sklearn.metrics.pairwise import euclidean_distances
            distances = euclidean_distances(query_vector, self.doc_vectors).flatten()
            similarities = -distances
        elif self.similarity_measure == 'manhattan':
            from sklearn.metrics.pairwise import manhattan_distances
            distances = manhattan_distances(query_vector, self.doc_vectors).flatten()
            similarities = -distances
        else:
            similarities = cosine_similarity(query_vector, self.doc_vectors).flatten()
        
        # Sort by similarity (descending)
        ranked_docs = [(i, sim) for i, sim in enumerate(similarities)]
        ranked_docs.sort(key=lambda x: x[1], reverse=True)
        
        return ranked_docs[:top_k]

In [116]:
class BM25Model:
    """BM25 Probabilistic Model"""
    
    def __init__(self, documents: List[str], processor: TextProcessor, 
                 k1: float = 1.2, b: float = 0.75, variant='standard'):
        self.documents = documents
        self.processor = processor
        self.k1 = k1
        self.b = b
        self.variant = variant
        
        # Preprocess documents
        self.processed_docs = []
        for doc in documents:
            tokens = processor.preprocess_text(doc)
            self.processed_docs.append(tokens)
        
        # Calculate document frequencies and lengths
        self.doc_freqs = []
        self.doc_lengths = []
        self.term_doc_freq = defaultdict(int)
        
        for tokens in self.processed_docs:
            doc_freq = Counter(tokens)
            self.doc_freqs.append(doc_freq)
            self.doc_lengths.append(len(tokens))
            
            # Count document frequency for each term
            for term in set(tokens):
                self.term_doc_freq[term] += 1
        
        self.avg_doc_length = sum(self.doc_lengths) / len(self.doc_lengths)
        self.corpus_size = len(documents)
    
    def _calculate_idf(self, term: str) -> float:
        """Calculate IDF for a term"""
        df = self.term_doc_freq.get(term, 0)
        if df == 0:
            return 0
        return math.log((self.corpus_size - df + 0.5) / (df + 0.5))
    
    def _calculate_score(self, query_terms: List[str], doc_id: int) -> float:
        """Calculate BM25 score for a document"""
        score = 0
        doc_freq = self.doc_freqs[doc_id]
        doc_length = self.doc_lengths[doc_id]
        
        for term in query_terms:
            if term not in doc_freq:
                continue
            
            tf = doc_freq[term]
            idf = self._calculate_idf(term)
            
            if self.variant == 'standard':
                # Standard BM25
                numerator = tf * (self.k1 + 1)
                denominator = tf + self.k1 * (1 - self.b + self.b * (doc_length / self.avg_doc_length))
                score += idf * (numerator / denominator)
            
            elif self.variant == 'bm25l':
                # BM25L variant
                delta = 0.5  # BM25L parameter
                numerator = tf * (self.k1 + 1)
                denominator = tf + self.k1 * (1 - self.b + self.b * (doc_length / self.avg_doc_length))
                score += idf * (numerator / denominator + delta)
            
            elif self.variant == 'bm25plus':
                # BM25+ variant
                delta = 0.8  # BM25+ parameter
                numerator = tf * (self.k1 + 1)
                denominator = tf + self.k1 * (1 - self.b + self.b * (doc_length / self.avg_doc_length))
                score += idf * (numerator / denominator + delta)
        
        return score
    
    def search(self, query: str, top_k: int = 10) -> List[Tuple[int, float]]:
        """Search using BM25"""
        query_terms = self.processor.preprocess_text(query)
        
        if not query_terms:
            return []
        
        # Calculate scores for all documents
        scores = []
        for doc_id in range(len(self.documents)):
            score = self._calculate_score(query_terms, doc_id)
            scores.append((doc_id, score))
        
        # Sort by score (descending)
        scores.sort(key=lambda x: x[1], reverse=True)
        
        return scores[:top_k]

In [117]:
print("\nInitializing models...")
processor = TextProcessor(use_stopwords=True, use_lemmatization=True)

boolean_model = BooleanModel(documents, processor)
vector_model = VectorSpaceModel(documents, processor, similarity_measure='cosine')
bm25_model = BM25Model(documents, processor, k1=1.2, b=0.75)


Initializing models...


In [118]:
# Test queries
queries = [
    "Earth's atmosphere",
    "Agricultural crops", 
    "Parts of the human body",
    "What are the official languages of countries?",
    "Best places to travel"
]

def display_results(query: str, boolean_results: List[int], 
                   vector_results: List[Tuple[int, float]], 
                   bm25_results: List[Tuple[int, float]]):
    """Display search results for comparison"""
    print(f"\n{'='*60}")
    print(f"Query: '{query}'")
    print(f"{'='*60}")
    
    print(f"\nBoolean Model Results ({len(boolean_results)} documents):")
    for i, doc_id in enumerate(boolean_results[:5]):
        print(f"{i+1}. [{doc_id}] {titles[doc_id]}")
    
    print(f"\nVector Space Model Results (Top 5):")
    for i, (doc_id, score) in enumerate(vector_results[:5]):
        print(f"{i+1}. [{doc_id}] {titles[doc_id]} (Score: {score:.4f})")
    
    print(f"\nBM25 Model Results (Top 5):")
    for i, (doc_id, score) in enumerate(bm25_results[:5]):
        print(f"{i+1}. [{doc_id}] {titles[doc_id]} (Score: {score:.4f})")

# Part 1 & 2: Run searches and compare results
print("\n" + "="*80)
print("PART 1 & 2: SEARCHING WITH DIFFERENT MODELS")
print("="*80)

for query in queries:
    boolean_results = boolean_model.search(query)
    vector_results = vector_model.search(query, top_k=10)
    bm25_results = bm25_model.search(query, top_k=10)
    
    display_results(query, boolean_results, vector_results, bm25_results)

# Test with longer queries
print("\n" + "="*80)
print("TESTING WITH LONGER QUERIES")
print("="*80)

longer_queries = [
    "Name parts of the human body",
    "What are agricultural crops used for food production",
    "Describe Earth's atmospheric composition and layers"
]

for query in longer_queries:
    boolean_results = boolean_model.search(query)
    vector_results = vector_model.search(query, top_k=10)
    bm25_results = bm25_model.search(query, top_k=10)
    
    display_results(query, boolean_results, vector_results, bm25_results)


PART 1 & 2: SEARCHING WITH DIFFERENT MODELS

Query: 'Earth's atmosphere'

Boolean Model Results (8 documents):
1. [129] Ecology
2. [4] Air
3. [139] February
4. [118] Earth science
5. [119] Earth

Vector Space Model Results (Top 5):
1. [4] Air (Score: 0.3555)
2. [119] Earth (Score: 0.3525)
3. [118] Earth science (Score: 0.3438)
4. [63] Classical element (Score: 0.1382)
5. [87] Crust (Score: 0.1275)

BM25 Model Results (Top 5):
1. [4] Air (Score: 8.7308)
2. [118] Earth science (Score: 8.2315)
3. [24] Astronomy (Score: 6.4363)
4. [153] Geography (Score: 5.9803)
5. [119] Earth (Score: 5.4769)

Query: 'Agricultural crops'

Boolean Model Results (4 documents):
1. [144] Food
2. [10] Farming
3. [43] Beekeeping
4. [54] Botany

Vector Space Model Results (Top 5):
1. [10] Farming (Score: 0.2180)
2. [43] Beekeeping (Score: 0.1129)
3. [54] Botany (Score: 0.1018)
4. [152] Farm (Score: 0.0626)
5. [144] Food (Score: 0.0355)

BM25 Model Results (Top 5):
1. [10] Farming (Score: 9.7554)
2. [43] Beekeepi

In [121]:
# Part 3: Test without lemmatization and stop words
print("\n" + "="*80)
print("PART 3: TESTING WITHOUT LEMMATIZATION AND STOP WORDS")
print("="*80)

# Initialize models without preprocessing
processor_no_preprocess = TextProcessor(use_stopwords=False, use_lemmatization=False)

boolean_model_no_preprocess = BooleanModel(documents, processor_no_preprocess)
vector_model_no_preprocess = VectorSpaceModel(documents, processor_no_preprocess)
bm25_model_no_preprocess = BM25Model(documents, processor_no_preprocess)

# Test with a sample query
test_query = 'What are agricultural crops used for food production'
print(f"\nComparing results for: '{test_query}'")

print("\nWITH preprocessing (lemmatization + stop word removal):")
boolean_results_with = boolean_model.search(test_query)
vector_results_with = vector_model.search(test_query, top_k=5)
bm25_results_with = bm25_model.search(test_query, top_k=5)

print(f"Boolean: {len(boolean_results_with)} results")
print("Vector Space Top 3:", [(titles[doc_id], f"{score:.4f}") for doc_id, score in vector_results_with[:3]])
print("BM25 Top 3:", [(titles[doc_id], f"{score:.4f}") for doc_id, score in bm25_results_with[:3]])

print("\nWITHOUT preprocessing:")
boolean_results_without = boolean_model_no_preprocess.search(test_query)
vector_results_without = vector_model_no_preprocess.search(test_query, top_k=5)
bm25_results_without = bm25_model_no_preprocess.search(test_query, top_k=5)

print(f"Boolean: {len(boolean_results_without)} results")
print("Vector Space Top 3:", [(titles[doc_id], f"{score:.4f}") for doc_id, score in vector_results_without[:3]])
print("BM25 Top 3:", [(titles[doc_id], f"{score:.4f}") for doc_id, score in bm25_results_without[:3]])


PART 3: TESTING WITHOUT LEMMATIZATION AND STOP WORDS

Comparing results for: 'What are agricultural crops used for food production'

WITH preprocessing (lemmatization + stop word removal):
Boolean: 1 results
Vector Space Top 3: [('Food', '0.2762'), ('Farming', '0.2410'), ('Cooking', '0.1816')]
BM25 Top 3: [('Farming', '15.0084'), ('Botany', '12.1522'), ('Food', '11.0247')]

WITHOUT preprocessing:
Boolean: 1 results
Vector Space Top 3: [('Food', '0.2515'), ('Farming', '0.2039'), ('Cooking', '0.1584')]
BM25 Top 3: [('Farming', '10.5733'), ('Food', '7.9084'), ('Botany', '7.3144')]


In [ ]:
# defining the preprocessing function

digits = re.compile(r'\d')
lemmatizer = WordNetLemmatizer()
def preprocess(doc):   #this takes in a string and converts it into a list
  doc = doc.split()
  preprocessed_text = []
  for text in doc:
    text = text.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    text = text.lower() #convert to lower case
    words = word_tokenize(text) #tokenize the text
    words = [word for word in words if word not in stopwords.words('english')] #remove stopwords
    words = [word for word in words if not digits.match(word)] # additional step : removing digits
    words = [lemmatizer.lemmatize(word) for word in words] #lemmatize
    if words != []:
      preprocessed_text.append(words[0])
  return preprocessed_text

In [ ]:
# let's define the query again:

# query = "Earth's atmosphere"
# query = "Agricultural crops"
# query = "Parts of the human body"
# query = "What are the official languages of countries?"
query = "Best places to travel as a tourist"

# query = 'name parts of the human body.'


# we will have to preprocess the query before we convert it into a vector:

query = preprocess(query)
print(query)

In [ ]:
# compute cosine similarities
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(query_vector, X)
results = [(data[i], cosine_similarities[0][i]) for i in range(len(data))]
results.sort(key=lambda x: x[1], reverse=True)
for doc, similarity in results[:5]:
    print(f"Similarity: {similarity:.2f}\n{doc}\n")

In [ ]:
# compute euclidean distance
from sklearn.metrics.pairwise import euclidean_distances

euclidean_dist = euclidean_distances(query_vector, X)
results = [(data[i], euclidean_dist[0][i]) for i in range(len(data))]
results.sort(key=lambda x: x[1])
for doc, similarity in results[:5]:
    print(f"Distance: {similarity:.2f}\n{doc}\n")

In [120]:
print("\n" + "="*80)
print("PART 4: DIFFERENT SIMILARITY MEASURES FOR VECTOR SPACE MODEL")
print("="*80)

# Initialize vector models with different similarity measures
vector_model_cosine = VectorSpaceModel(documents, processor, similarity_measure='cosine')
vector_model_euclidean = VectorSpaceModel(documents, processor, similarity_measure='euclidean')
vector_model_manhattan = VectorSpaceModel(documents, processor, similarity_measure='manhattan')

test_query = "Agricultural crops"
print(f"\nComparing similarity measures for: '{test_query}'")

cosine_results = vector_model_cosine.search(test_query, top_k=5)
euclidean_results = vector_model_euclidean.search(test_query, top_k=5)
manhattan_results = vector_model_manhattan.search(test_query, top_k=5)

print("\nCosine Similarity:")
for i, (doc_id, score) in enumerate(cosine_results):
    print(f"{i+1}. {titles[doc_id]} (Score: {score:.4f})")

print("\nEuclidean Distance (negative):")
for i, (doc_id, score) in enumerate(euclidean_results):
    print(f"{i+1}. {titles[doc_id]} (Score: {score:.4f})")

print("\nManhattan Distance (negative):")
for i, (doc_id, score) in enumerate(manhattan_results):
    print(f"{i+1}. {titles[doc_id]} (Score: {score:.4f})")

# Extra: BM25 variants and parameter tuning
print("\n" + "="*80)
print("EXTRA: BM25 VARIANTS AND PARAMETER TUNING")
print("="*80)

# Test BM25 variants
bm25_standard = BM25Model(documents, processor, k1=1.2, b=0.75, variant='standard')
bm25l = BM25Model(documents, processor, k1=1.2, b=0.75, variant='bm25l')
bm25plus = BM25Model(documents, processor, k1=1.2, b=0.75, variant='bm25plus')

test_query = "Earth's atmosphere"
print(f"\nComparing BM25 variants for: '{test_query}'")

standard_results = bm25_standard.search(test_query, top_k=5)
bm25l_results = bm25l.search(test_query, top_k=5)
bm25plus_results = bm25plus.search(test_query, top_k=5)

print("\nStandard BM25:")
for i, (doc_id, score) in enumerate(standard_results):
    print(f"{i+1}. {titles[doc_id]} (Score: {score:.4f})")

print("\nBM25L:")
for i, (doc_id, score) in enumerate(bm25l_results):
    print(f"{i+1}. {titles[doc_id]} (Score: {score:.4f})")

print("\nBM25+:")
for i, (doc_id, score) in enumerate(bm25plus_results):
    print(f"{i+1}. {titles[doc_id]} (Score: {score:.4f})")

# Parameter tuning for BM25
print(f"\nTesting different BM25 parameters for: '{test_query}'")

# Test different k1 values
k1_values = [0.5, 1.2, 2.0]
b_values = [0.25, 0.75, 1.0]

print("\nParameter tuning results (showing top result for each combination):")
for k1 in k1_values:
    for b in b_values:
        bm25_tuned = BM25Model(documents, processor, k1=k1, b=b)
        results = bm25_tuned.search(test_query, top_k=1)
        if results:
            doc_id, score = results[0]
            print(f"k1={k1}, b={b}: {titles[doc_id]} (Score: {score:.4f})")

print("\n" + "="*80)
print("ANALYSIS COMPLETE - ALL PARTS IMPLEMENTED")
print("="*80)



PART 4: DIFFERENT SIMILARITY MEASURES FOR VECTOR SPACE MODEL

Comparing similarity measures for: 'Agricultural crops'

Cosine Similarity:
1. Farming (Score: 0.2180)
2. Beekeeping (Score: 0.1129)
3. Botany (Score: 0.1018)
4. Farm (Score: 0.0626)
5. Food (Score: 0.0355)

Euclidean Distance (negative):
1. Farming (Score: -1.2506)
2. Beekeeping (Score: -1.3320)
3. Botany (Score: -1.3403)
4. Farm (Score: -1.3692)
5. Food (Score: -1.3889)

Manhattan Distance (negative):
1. Contact network (Score: -3.3968)
2. Ewe (Score: -3.4567)
3. Compound (Score: -3.7846)
4. Italian (Score: -3.8409)
5. ISO 19011 (Score: -4.0420)

EXTRA: BM25 VARIANTS AND PARAMETER TUNING

Comparing BM25 variants for: 'Earth's atmosphere'

Standard BM25:
1. Air (Score: 8.7308)
2. Earth science (Score: 8.2315)
3. Astronomy (Score: 6.4363)
4. Geography (Score: 5.9803)
5. Earth (Score: 5.4769)

BM25L:
1. Air (Score: 11.0436)
2. Earth science (Score: 10.5443)
3. Astronomy (Score: 8.7492)
4. Geography (Score: 8.2931)
5. Earth (